In [1]:
import pandas as pd
import numpy as np

In [11]:
# Read the excel file sent by Moy Hin
file_path = r"C:\Users\Gabriel\Documents\Motiro\Asia-Pacific_Volunteer_Survey_2025-05-10_Motiro.xlsx"
df = pd.read_excel(file_path)
# Display the last rows of the dataframe to verify successful loading
df.tail()

# STEP 1.1 : recoding answers from string to integer in order to compute scores and means

# Define the mapping of categorical values to integers
mapping = {
    'Strongly disagree': 1,
    'Somewhat disagree': 2,
    'Neutral': 3,
    'Somewhat agree': 4,
    'Strongly agree': 5
}

# transform the categorical column into integers using the mapping from column 10 to column 33
# 10 to 33 = 24 columns 

# Find all columns that match the mapping keys (i.e., contain Likert scale answers)
likert_columns = [col for col in df.columns if set(mapping.keys()) & set(df[col].unique()) or any(isinstance(x, str) and x in mapping for x in df[col].unique())]

for col in likert_columns:
    df[col] = df[col].replace(mapping)
    df[col] = pd.to_numeric(df[col], errors='coerce')

#replace string values with real numbers 1-7 to compute means for frequency variable, by teams
# At my volunteer activity, I feel strong and vigorous      column[34]     
# I feel emotionally drained from my work.                  column[35]     
# I feel frustrated by my work.                             column[36]  

mapping = {
    "Every time": 7,
    "Very often": 6,
    "Often": 5,
    "Sometimes": 4,
    "Rarely": 3,
    "Almost never": 2,
    "Never": 1
} 

df['At my volunteer activity, I feel strong and vigorous.'] = df['At my volunteer activity, I feel strong and vigorous.'].replace(mapping)
df['I feel emotionally drained from my volunteer activity.'] = df['I feel emotionally drained from my volunteer activity.'].replace(mapping)
df['I feel frustrated by my volunteer activity.'] = df['I feel frustrated by my volunteer activity.'].replace(mapping)

df.head()

# STEP 1.2 : Create the scores for each dimension, e.g. autonomy, belonging, etc.
#  Before computing the well-being score: 
#  recode the two following columns so that the positive outcomes get the higher scores
# 'I feel emotionally drained from my volunteer activity.
# 'I feel frustrated by my volunteer activity.

mapping = {
    7: 1,
    6: 2,
    5: 3,
    4: 4,
    3: 5,
    2: 6,
    1: 7
}

df['I feel emotionally drained from my volunteer activity.'] = df['I feel emotionally drained from my volunteer activity.'].replace(mapping)
df['I feel frustrated by my volunteer activity.'] = df['I feel frustrated by my volunteer activity.'].replace(mapping)

# Specify the column names for Well-being
column_names = [
    'At my volunteer activity, I feel strong and vigorous.',
    'I feel emotionally drained from my volunteer activity.',
    'I feel frustrated by my volunteer activity.'
]

# Calculate the mean row by row and assign it to a new column 'Well-being'
df['Well-being'] = df[column_names].mean(axis=1)

# Rescale the 'Well-being' values from a range of 1-7 to 1-5 and rounds the result
df['Well-being'] = (df['Well-being'] - 1) * (4 / 6) + 1
df['Well-being'] = df['Well-being'].round(1)

## recode back to original values for the two "negative" variables
#  after computing the well-being score for "logical" correlation analysis
#  we need to recode the two following columns so that the negative outcomes get the higher scores
# 'I feel emotionally drained from my volunteer activity.
# 'I feel frustrated by my volunteer activity.

mapping = {
    7: 1,
    6: 2,
    5: 3,
    4: 4,
    3: 5,
    2: 6,
    1: 7
}

df['I feel emotionally drained from my volunteer activity.'] = df['I feel emotionally drained from my volunteer activity.'].replace(mapping)
df['I feel frustrated by my volunteer activity.'] = df['I feel frustrated by my volunteer activity.'].replace(mapping)


# Specify the column names for Engagement
column_names = [
    'If I could choose, I will be volunteering one year from now.',
    'The team has a great deal of personal meaning for me.',
    'Considering everything, I am satisfied with my volunteer activity.',
    'I share my ideas with others to improve the team.',
    'The things that I value in life are very similar to the things that the team values.'
]

# Calculate the mean row by row and assign it to a new column 'Engagement'
df['Engagement'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for Autonomy
column_names = [
    'I feel like I can be myself at my volunteer activity.',
    'The tasks I do at my volunteer activity are in line with what I really want to do.',
    'I am free to express my ideas and opinions on the volunteer activity.'
]

# Calculate the mean row by row and assign it to a new column 'Autonomy'
df['Autonomy'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for belonging
column_names = [
    'At my volunteer activity, I feel part of a group.',
    'At my volunteer activity, I can talk with people about things that really matter to me.',
    'People at my volunteer activity care about me.'
]

# Calculate the mean row by row and assign it to a new column 'Belonging'
df['Belonging'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for Competence
column_names = [
    'I really master my tasks at my volunteer activity.',
    'I feel competent at my volunteer activity.'
]

# Calculate the mean row by row and assign it to a new column 'Competence'
df['Competence'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for Needs
column_names = [
    'I feel like I can be myself at my volunteer activity.',
    'The tasks I do at my volunteer activity are in line with what I really want to do.',
    'I am free to express my ideas and opinions on the volunteer activity.',
    'At my volunteer activity, I feel part of a group.',
    'At my volunteer activity, I can talk with people about things that really matter to me.',
    'People at my volunteer activity care about me.',
    'I really master my tasks at my volunteer activity.',
    'I feel competent at my volunteer activity.'
]

# Calculate the mean row by row and assign it to a new column 'Needs'
df['Needs'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for Leadership
column_names = [
    'My supervisor listens to how I would like to do things.',
    'I feel understood by my supervisor.',
    'My supervisor encourages me to ask questions.'
]

# Calculate the mean row by row and assign it to a new column 'Leadership'
df['Leadership'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for 'Management'
column_names = [
    'I have the opportunity to develop my social network in my volunteer activity.',
    'I have been able to learn interesting new skills on my volunteer activity.',
    'I am fairly rewarded considering the responsibilities I have.',
    'I am fairly rewarded for the work I do well.',
    'My family, friends and my neighborhood appreciate the work I do for the team.',
    'I feel my work has positive impact on other people.'
]

# Calculate the mean row by row and assign it to a new column 'Management'
df['Management'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for 'Returns'
column_names = [
    'I have the opportunity to develop my social network in my volunteer activity.',
    'I have been able to learn interesting new skills on my volunteer activity.'
]

# Calculate the mean row by row and assign it to a new column 'Returns'
df['Returns'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for 'Rewards'
column_names = [
    'I am fairly rewarded considering the responsibilities I have.',
    'I am fairly rewarded for the work I do well.'
]

# Calculate the mean row by row and assign it to a new column 'Rewards'
df['Rewards'] = df[column_names].mean(axis=1).round(1)

# Specify the column names for 'Status'
column_names = [
    'My family, friends and my neighborhood appreciate the work I do for the team.',
    'I feel my work has positive impact on other people.'
]

# Calculate the mean row by row and assign it to a new column 'Status'
df['Status'] = df[column_names].mean(axis=1).round(1)


# STEP 1.3 : Rename columns 10 to 36 in their abridged form
# Define the new column names
# The first 10 columns are not changed, so we keep them as is   
# The next columns are renamed according to the provided list
cols_to_update = [
    'Myself',
    'Tasks',
    'Free',
    'Mastery',
    'Competent',
    'Learn',
    'Group',
    'Talk',
    'Care',
    'Satisfied',
    'Meaning',
    'Stay',
    'Ideas',
    'Values',
    'Understands',
    'Encourages',
    'Listens',
    'Network',
    'Friendly',
    'Team',
    'Appreciated',
    'Responsibilities',
    'Work',
    'Impact',
    'Strong',
    'Drained',
    'Frustrated'
    ]
# Make sure the number of columns matches the number of new names
num_cols_to_rename = len(cols_to_update)
# Create a dictionary to map old column names to new ones
new_column_names = {df.columns[i]: cols_to_update[i-10] for i in range(10, 10 + num_cols_to_rename)}
# Update the DataFrame's columns
df.rename(columns=new_column_names, inplace=True)
# Display the updated DataFrame to verify the changes


# STEP 2 : Replace column names with their abbreviations
# Define the mapping of full column names to their abbreviations
abbreviation_mapping = {
    'How long have you been volunteering for the Red Cross Red Crescent National Society?': 'How long?',
    'What is your age group?': 'Age',
    'Which Red Cross Red Crescent National Society do you belong?': 'NS',
    'Are you also volunteering elsewhere?': 'Elsewhere',
    'When was the last time you joined a Red Cross Red Crescent National Society activity?': 'Last time',
    'Have you volunteered for other organisations before?': 'Other org',
    'Do you volunteer in urban or rural area or both?': 'Urban/Rural',
    'What is your gender?': 'Gender',
    'What is your highest level of education?': 'Education',
    'Which of the following best describes your current employment status?': 'Employment',
    'Do you identify yourself as a person forcedly displaced from your home due to crisis?': 'Displaced',
}
# Replace the full column names with their abbreviations
df.rename(columns=abbreviation_mapping, inplace=True)   

# Display the updated DataFrame to verify the changes
df.head()

df.info()
df.to_csv('AsiaPacific_ALL.csv', index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7637 entries, 0 to 7636
Columns: 236 entries, _id to Status
dtypes: datetime64[ns](1), float64(62), int64(84), object(89)
memory usage: 13.8+ MB


In [ ]:
# STEP 3: create dataframe where one row is one NS
df = pd.read_csv('AsiaPacific_ALL.csv')

#Select the columns to keep
columns_to_keep = [
    'NS',
    'How long?',
    'Age',
    'Gender',
    'Elsewhere',
    'Last time',
    'Other org',
    'Urban/Rural',
    'Education',
    'Employment',
    'Displaced',
    'Myself',
    'Tasks',
    'Free',
    'Mastery',
    'Competent',
    'Learn',
    'Group',
    'Talk',
    'Care',
    'Satisfied',
    'Meaning',
    'Stay',
    'Ideas',
    'Values',
    'Understands',
    'Encourages',
    'Listens',
    'Network',
    'Friendly',
    'Team',
    'Appreciated',
    'Responsibilities',
    'Work',
    'Impact',
    'Strong',
    'Drained',
    'Frustrated',
    'Well-being',
    'Engagement',
    'Autonomy',
    'Belonging',
    'Competence',
    'Needs',
    'Leadership',
    'Management',
    'Returns',
    'Rewards',
    'Status'
]

# Create a new DataFrame with only the selected columns
df = df[columns_to_keep]

# Display the updated DataFrame to verify the changes
df.info()

# STEP 3.5  Save DataFrame
df.to_csv('AsiaPacific_short_ALL.csv', index=False)

# STEP 4 : Create a DataFrame with the mean and median for each NS
# Create list of columns for which to compute the mean and median for each NS
mean_columns = [
    'Myself',
    'Tasks',
    'Free',
    'Mastery',
    'Competent',
    'Learn',
    'Group',
    'Talk',
    'Care',
    'Satisfied',
    'Meaning',
    'Stay',
    'Ideas',
    'Values',
    'Understands',
    'Encourages',
    'Listens',
    'Network',
    'Friendly',
    'Team',
    'Appreciated',
    'Responsibilities',
    'Work',
    'Impact',
    'Strong',
    'Drained',
    'Frustrated',
    'Well-being',
    'Engagement',
    'Autonomy',
    'Belonging',
    'Competence',
    'Needs',
    'Leadership',
    'Management',
    'Returns',
    'Rewards',
    'Status'
]

# STEP 3.1 Group rows by 'NS' and calculate size (count) for each NS
mean_df = df.groupby('NS')[mean_columns].mean().round(1).reset_index()
median_df = df.groupby('NS')[mean_columns].median().reset_index()
# Display the mean and median DataFrames
mean_df.info()
median_df.info()

# STEP 3.2  Merge the mean and median DataFrames
df = df.merge(mean_df, on='NS', suffixes=('_mean', '_median')).merge(median_df, on='NS', suffixes=('_mean', '_median'))
# Display the merged DataFrame
df.info()
# STEP 3.3  Create a DataFrame with the size (count) for each NS
NS_df = df.groupby('NS').size().reset_index(name='count')
# Display the size DataFrame
NS_df.info()

# STEP 3.4 Merge the size DataFrame with the mean and median DataFrames 
NS_df = NS_df.merge(mean_df, on='NS', suffixes=('_mean', '_median')).merge(median_df, on='NS', suffixes=('_mean', '_median'))
# Display the merged DataFrame
NS_df.info()

# STEP 3.5  Save DataFrame
NS_df.to_csv('AsiaPacific_NS.csv', index=False)



C:\Users\Gabriel\AppData\Local\Temp\ipykernel_11880\249525120.py:2: DtypeWarning: Columns (189,190,208,209,210,211,212,213) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('AsiaPacific_ALL.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7637 entries, 0 to 7636
Data columns (total 49 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   NS                7637 non-null   object 
 1   How long?         7637 non-null   object 
 2   Age               7637 non-null   object 
 3   Gender            7637 non-null   object 
 4   Elsewhere         7637 non-null   object 
 5   Last time         7637 non-null   object 
 6   Other org         7637 non-null   object 
 7   Urban/Rural       7637 non-null   object 
 8   Education         7637 non-null   object 
 9   Employment        7637 non-null   object 
 10  Displaced         7637 non-null   object 
 11  Myself            7636 non-null   float64
 12  Tasks             7636 non-null   float64
 13  Free              7636 non-null   float64
 14  Mastery           7636 non-null   float64
 15  Competent         7636 non-null   float64
 16  Learn             7636 non-null   float64


In [17]:
# Create cross tabulations of Moti scores and key demographic variables
# Read the CSV file
df = pd.read_csv('AsiaPacific_short_ALL.csv')

#Create list of columns containing demographic data for tabulation
demographic_columns = [
    'How long?',
    'Age',
    'NS',
    'Elsewhere',
    'Last time',
    'Other org',
    'Urban/Rural',
    'Education',
    'Employment',
    'Displaced',
    'Gender',
]

# Create list of columns containing Moti scores for tabulation
moti_columns = [
    'Myself',
    'Tasks',
    'Free',
    'Mastery',
    'Competent',
    'Learn',
    'Group',
    'Talk',
    'Care',
    'Satisfied',
    'Meaning',
    'Stay',
    'Ideas',
    'Values',
    'Understands',
    'Encourages',
    'Listens',
    'Network',
    'Friendly',
    'Team',
    'Appreciated',
    'Responsibilities',
    'Work',
    'Impact',
]

# Create cross tabulations of demographic variables and Moti scores
for demographic in demographic_columns:
    for moti in moti_columns:
        crosstab = pd.crosstab(df[demographic], df[moti], margins=True, margins_name="Total")
        # Display the cross tabulation
        print(f"Cross tabulation of {demographic} and {moti}:")
        print(crosstab)
        print("\n")



Cross tabulation of How long? and Myself:
Myself              1.0  2.0   3.0   4.0   5.0  Total
How long?                                            
1-3 years            41   27   398   494  1544   2504
4-5 years            16   12   141   165   694   1028
6-10 years           13   14   114   147   546    834
Less than 1 year     43   36   594   420  1209   2302
More than 10 years   23   17   112   172   644    968
Total               136  106  1359  1398  4637   7636


Cross tabulation of How long? and Tasks:
Tasks               1.0  2.0   3.0   4.0   5.0  Total
How long?                                            
1-3 years            42   46   360   572  1484   2504
4-5 years            18   14   121   248   627   1028
6-10 years           18   15    99   220   482    834
Less than 1 year     30   32   549   491  1200   2302
More than 10 years   22   18   106   279   543    968
Total               130  125  1235  1810  4336   7636


Cross tabulation of How long? and Free:
Free     